In [3]:
# folders

project_dir1 = "2024-09-21_18-59-52_B2C_2024_constrained_3prompts_250maxtokens"
project_dir2 = "2024-09-23_13-13-43_B2C_2024_constrained_3prompts_250maxtokens_gemmabase"

#project_dir3 = "2024-10-01_09-24-20_B2C_2024_constrained_3prompts_250maxtokens_llama3" # this run was a mistake

project_dir3 = "2024-10-01_11-30-25_B2C_2024_constrained_3prompts_250maxtokens_llama3"

project_dir4 = "2024-10-08_14-20-12_B2C_2024_constrained_3prompts_250maxtokens_mistralv7b0.1"
project_dir5 = "2024-10-09_06-05-37_B2C_2024_constrained_3prompts_250maxtokens_llama2"


# Individual predictions


In [4]:
import pandas as pd


In [5]:
# Load aggregated metrics for each run condition
df1 = pd.read_hdf(project_dir1+"/predictions.h5", key='predictions')
df1["project_dir"] = project_dir1

df2 = pd.read_hdf(project_dir2+"/predictions.h5", key='predictions')
df2["project_dir"] = project_dir2

df3 = pd.read_hdf(project_dir3+"/predictions.h5", key='predictions')
df3["project_dir"] = project_dir3

df4 = pd.read_hdf(project_dir4+"/predictions.h5", key='predictions')
df4["project_dir"] = project_dir4

df5 = pd.read_hdf(project_dir5+"/predictions.h5", key='predictions')
df5["project_dir"] = project_dir5

df = pd.concat([df1, df2, df3, df4, df5])
df = df.loc[df.prompt == "full_description"]

In [7]:
individual_models = df.model.unique().tolist()
individual_models

['RuterNorway/Llama-2-7b-chat-norwegian',
 'RuterNorway/Llama-2-13b-chat-norwegian',
 'norallm/normistral-7b-warm-instruct',
 'norallm/normistral-7b-warm',
 'norallm/normistral-7b-scratch',
 'NorwAI/NorwAI-Mistral-7B-instruct',
 'NorwAI/NorwAI-Mistral-7B',
 'bineric/NorskGPT-Llama3-8b',
 'bineric/NorskGPT-Mistral-7b',
 'mistralai/Mistral-7B-v0.1',
 'meta-llama/Meta-Llama-3.1-8B',
 'meta-llama/Meta-Llama-3.1-8B-Instruct',
 'google/gemma-2-2b-it',
 'google/gemma-2-9b-it',
 'google/gemma-2-27b-it',
 'google/gemma-2-2b',
 'google/gemma-2-9b',
 'google/gemma-2-27b',
 'meta-llama/Meta-Llama-3-8B',
 'meta-llama/Meta-Llama-3-8B-Instruct',
 'mistralai/Mistral-7B-Instruct-v0.1',
 'meta-llama/Llama-2-7b-chat-hf',
 'meta-llama/Llama-2-13b-chat-hf']

In [8]:
df.model.value_counts()


model
RuterNorway/Llama-2-7b-chat-norwegian     3000
google/gemma-2-2b-it                      3000
meta-llama/Llama-2-7b-chat-hf             3000
mistralai/Mistral-7B-Instruct-v0.1        3000
meta-llama/Meta-Llama-3-8B-Instruct       3000
meta-llama/Meta-Llama-3-8B                3000
google/gemma-2-27b                        3000
google/gemma-2-9b                         3000
google/gemma-2-2b                         3000
google/gemma-2-27b-it                     3000
google/gemma-2-9b-it                      3000
meta-llama/Meta-Llama-3.1-8B-Instruct     3000
RuterNorway/Llama-2-13b-chat-norwegian    3000
meta-llama/Meta-Llama-3.1-8B              3000
mistralai/Mistral-7B-v0.1                 3000
bineric/NorskGPT-Mistral-7b               3000
bineric/NorskGPT-Llama3-8b                3000
NorwAI/NorwAI-Mistral-7B                  3000
NorwAI/NorwAI-Mistral-7B-instruct         3000
norallm/normistral-7b-scratch             3000
norallm/normistral-7b-warm                3000
norallm

In [9]:
df.columns.tolist()

['replication',
 'server',
 'model',
 'model_type',
 'chat_format',
 'prompt',
 'dataset',
 'task_id',
 'duration_sec',
 'vram_used_gb',
 'ram_used_gb',
 'raw_tokens',
 'n_trunc_tokens',
 'input_text',
 'prompt_text',
 'predicted_label',
 'actual_label',
 'RUN',
 'project_dir']

In [10]:
import numpy as np
from sklearn.metrics import accuracy_score

# Calculate accuracy statistics for each model
models = []
replications = []
accuracies = []
batchnames = []
n_samples_per_rep = []

run_idx = 0
runs = []

for batchname in df.project_dir.unique():
    # get the subset for the current batch
    batch = df[df.project_dir == batchname]
    print(batchname)

    # for each model in the current batch
    for model in batch.model.unique():
        # subset on the current model
        experiment = batch[batch.model == model]

        for rep in experiment.replication.unique():
            # get the subset for the current replication
            subset_rep = experiment[experiment.replication == rep]

            # get the accuracy for the current replication
            accuracy = accuracy_score(subset_rep.actual_label, subset_rep.predicted_label)
            #print(f"Replication {rep}: Accuracy = {accuracy}")

            # append the accuracy to the list
            accuracies.append(accuracy)
            models.append(model)
            replications.append(rep)
            batchnames.append(batchname)
            n_samples_per_rep.append(len(subset_rep))
            runs.append(run_idx)


        run_idx += 1

results_df = pd.DataFrame({
        'Model': models,
        'Accuracy': accuracies,
        'Replication': replications,
        'Batchname': batchnames,
        'N_samples_per_rep': n_samples_per_rep,
        'Run': runs
    })

results_df

2024-09-21_18-59-52_B2C_2024_constrained_3prompts_250maxtokens
2024-09-23_13-13-43_B2C_2024_constrained_3prompts_250maxtokens_gemmabase
2024-10-01_11-30-25_B2C_2024_constrained_3prompts_250maxtokens_llama3
2024-10-08_14-20-12_B2C_2024_constrained_3prompts_250maxtokens_mistralv7b0.1
2024-10-09_06-05-37_B2C_2024_constrained_3prompts_250maxtokens_llama2


,Model,Accuracy,Replication,Batchname,N_samples_per_rep,Run
0,RuterNorway/Llama-2-7b-chat-norwegian,0.110000,0,2024-09-21_18-59-52_B2C_2024_constrained_3prom...,300,0
1,RuterNorway/Llama-2-7b-chat-norwegian,0.076667,1,2024-09-21_18-59-52_B2C_2024_constrained_3prom...,300,0
2,RuterNorway/Llama-2-7b-chat-norwegian,0.093333,2,2024-09-21_18-59-52_B2C_2024_constrained_3prom...,300,0
3,RuterNorway/Llama-2-7b-chat-norwegian,0.093333,3,2024-09-21_18-59-52_B2C_2024_constrained_3prom...,300,0
4,RuterNorway/Llama-2-7b-chat-norwegian,0.103333,4,2024-09-21_18-59-52_B2C_2024_constrained_3prom...,300,0
...,...,...,...,...,...,...
225,meta-llama/Llama-2-13b-chat-hf,0.083333,5,2024-10-09_06-05-37_B2C_2024_constrained_3prom...,300,22
226,meta-llama/Llama-2-13b-chat-hf,0.090000,6,2024-10-09_06-05-37_B2C_2024_constrained_3prom...,300,22
227,meta-llama/Llama-2-13b-chat-hf,0.093333,7,2024-10-09_06-05-37_B2C_2024_constrained_3prom...,300,22
228,meta-llama/Llama-2-13b-chat-hf,0.090000,8,2024-10-09_06-05-37_B2C_2024_constrained_3prom...,300,22


## add macro f1 and mcc

In [11]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef

# Calculate accuracy statistics for each model
models = []
replications = []
accuracies = []
macro_f1s = []  # New list for macro-F1 scores
mccs = []       # New list for MCC scores
batchnames = []
n_samples_per_rep = []

run_idx = 0
runs = []

for batchname in df.project_dir.unique():
    # get the subset for the current batch
    batch = df[df.project_dir == batchname]
    print(batchname)

    # for each model in the current batch
    for model in batch.model.unique():
        # subset on the current model
        experiment = batch[batch.model == model]

        for rep in experiment.replication.unique():
            # get the subset for the current replication
            subset_rep = experiment[experiment.replication == rep]

            # get the metrics for the current replication
            accuracy = accuracy_score(subset_rep.actual_label, subset_rep.predicted_label)
            macro_f1 = f1_score(subset_rep.actual_label, subset_rep.predicted_label, average='macro')
            mcc = matthews_corrcoef(subset_rep.actual_label, subset_rep.predicted_label)

            # append the metrics to the lists
            accuracies.append(accuracy)
            macro_f1s.append(macro_f1)
            mccs.append(mcc)
            models.append(model)
            replications.append(rep)
            batchnames.append(batchname)
            n_samples_per_rep.append(len(subset_rep))
            runs.append(run_idx)

        run_idx += 1

results_df = pd.DataFrame({
        'Run': runs,
        'Model': models,
        'Accuracy': accuracies,
        'Macro_F1': macro_f1s,
        'MCC': mccs,
        'Replication': replications,
        'Batchname': batchnames,
        'N_samples': n_samples_per_rep
    })


2024-09-21_18-59-52_B2C_2024_constrained_3prompts_250maxtokens
2024-09-23_13-13-43_B2C_2024_constrained_3prompts_250maxtokens_gemmabase
2024-10-01_11-30-25_B2C_2024_constrained_3prompts_250maxtokens_llama3
2024-10-08_14-20-12_B2C_2024_constrained_3prompts_250maxtokens_mistralv7b0.1
2024-10-09_06-05-37_B2C_2024_constrained_3prompts_250maxtokens_llama2


In [12]:
results_df

,Run,Model,Accuracy,Macro_F1,MCC,Replication,Batchname,N_samples
0,0,RuterNorway/Llama-2-7b-chat-norwegian,0.110000,0.086561,0.019310,0,2024-09-21_18-59-52_B2C_2024_constrained_3prom...,300
1,0,RuterNorway/Llama-2-7b-chat-norwegian,0.076667,0.064123,-0.014177,1,2024-09-21_18-59-52_B2C_2024_constrained_3prom...,300
2,0,RuterNorway/Llama-2-7b-chat-norwegian,0.093333,0.074088,0.025923,2,2024-09-21_18-59-52_B2C_2024_constrained_3prom...,300
3,0,RuterNorway/Llama-2-7b-chat-norwegian,0.093333,0.067452,0.013570,3,2024-09-21_18-59-52_B2C_2024_constrained_3prom...,300
4,0,RuterNorway/Llama-2-7b-chat-norwegian,0.103333,0.076782,0.018998,4,2024-09-21_18-59-52_B2C_2024_constrained_3prom...,300
...,...,...,...,...,...,...,...,...
225,22,meta-llama/Llama-2-13b-chat-hf,0.083333,0.051163,-0.006759,5,2024-10-09_06-05-37_B2C_2024_constrained_3prom...,300
226,22,meta-llama/Llama-2-13b-chat-hf,0.090000,0.062541,-0.026294,6,2024-10-09_06-05-37_B2C_2024_constrained_3prom...,300
227,22,meta-llama/Llama-2-13b-chat-hf,0.093333,0.083634,0.023295,7,2024-10-09_06-05-37_B2C_2024_constrained_3prom...,300
228,22,meta-llama/Llama-2-13b-chat-hf,0.090000,0.063708,0.001198,8,2024-10-09_06-05-37_B2C_2024_constrained_3prom...,300


In [13]:
results_df.to_excel("results_11_01_2025.xlsx")

# Aggregate it

In [14]:
import numpy as np

def compute_interval_radius(raw_scores: list[float]) -> float:
    """Compute the radius of a 95% confidence interval.

    Args:
        raw_scores:
            The scores that we need to compute the interval for.

    Returns:
        The radius of the interval.
    """
    sample_std_dev = np.std(raw_scores, ddof=1)
    std_err = sample_std_dev / np.sqrt(len(raw_scores))
    return 1.96 * std_err

In [15]:
decimals = 4

results_list = []

for run in results_df.Run.unique():
    subset = results_df[results_df.Run == run]
    
    res = {"run":run,
          "model":subset.Model.unique().tolist()[0],
          "mean_acc":np.round(np.mean(subset.Accuracy),decimals),
          "mean_acc_ci":np.round(compute_interval_radius(subset.Accuracy),decimals),
          "mean_Macro_F1":np.round(np.mean(subset.Macro_F1),decimals),
          "mean_Macro_F1_ci":np.round(compute_interval_radius(subset.Macro_F1),decimals),        
          "mean_MCC":np.round(np.mean(subset.MCC),decimals),
          "mean_MCC_ci":np.round(compute_interval_radius(subset.MCC),decimals)
    }   
    results_list.append(res)

agg_results = pd.DataFrame(results_list)
agg_results


,run,model,mean_acc,mean_acc_ci,mean_Macro_F1,mean_Macro_F1_ci,mean_MCC,mean_MCC_ci
0,0,RuterNorway/Llama-2-7b-chat-norwegian,0.1080,0.0114,0.0869,0.0129,0.0280,0.0151
1,1,RuterNorway/Llama-2-13b-chat-norwegian,0.2327,0.0198,0.1979,0.0110,0.1459,0.0205
2,2,norallm/normistral-7b-warm-instruct,0.1830,0.0134,0.0972,0.0083,0.0131,0.0125
3,3,norallm/normistral-7b-warm,0.1783,0.0129,0.1376,0.0127,0.0231,0.0154
4,4,norallm/normistral-7b-scratch,0.1647,0.0109,0.1389,0.0141,0.0199,0.0151
5,5,NorwAI/NorwAI-Mistral-7B-instruct,0.1717,0.0169,0.1355,0.0149,0.0258,0.0167
6,6,NorwAI/NorwAI-Mistral-7B,0.1683,0.0119,0.1255,0.0124,0.0122,0.0146
7,7,bineric/NorskGPT-Llama3-8b,0.1687,0.0151,0.1514,0.0150,0.0614,0.0190
8,8,bineric/NorskGPT-Mistral-7b,0.3050,0.0091,0.2689,0.0105,0.2254,0.0147
9,9,mistralai/Mistral-7B-v0.1,0.1287,0.0074,0.1031,0.0073,-0.0020,0.0062


In [16]:
agg_results.model.unique().tolist()


['RuterNorway/Llama-2-7b-chat-norwegian',
 'RuterNorway/Llama-2-13b-chat-norwegian',
 'norallm/normistral-7b-warm-instruct',
 'norallm/normistral-7b-warm',
 'norallm/normistral-7b-scratch',
 'NorwAI/NorwAI-Mistral-7B-instruct',
 'NorwAI/NorwAI-Mistral-7B',
 'bineric/NorskGPT-Llama3-8b',
 'bineric/NorskGPT-Mistral-7b',
 'mistralai/Mistral-7B-v0.1',
 'meta-llama/Meta-Llama-3.1-8B',
 'meta-llama/Meta-Llama-3.1-8B-Instruct',
 'google/gemma-2-2b-it',
 'google/gemma-2-9b-it',
 'google/gemma-2-27b-it',
 'google/gemma-2-2b',
 'google/gemma-2-9b',
 'google/gemma-2-27b',
 'meta-llama/Meta-Llama-3-8B',
 'meta-llama/Meta-Llama-3-8B-Instruct',
 'mistralai/Mistral-7B-Instruct-v0.1',
 'meta-llama/Llama-2-7b-chat-hf',
 'meta-llama/Llama-2-13b-chat-hf']

In [17]:
model_order = ["google/gemma-2-27b-it",
         "google/gemma-2-2b-it",
         "google/gemma-2-9b-it",
         "meta-llama/Llama-2-13b-chat-hf",
         "meta-llama/Llama-2-7b-chat-hf", 
         "meta-llama/Meta-Llama-3-8B-Instruct",
         "meta-llama/Meta-Llama-3.1-8B-Instruct",
         "mistralai/Mistral-7B-Instruct-v0.1",
         
         "RuterNorway/Llama-2-13b-chat-norwegian",
         "RuterNorway/Llama-2-7b-chat-norwegian",
         "bineric/NorskGPT-Llama3-8b",
         "bineric/NorskGPT-Mistral-7b",
         
         "google/gemma-2-27b",
         "google/gemma-2-2b",
         "google/gemma-2-9b",
         
         "meta-llama/Meta-Llama-3-8B",
         "meta-llama/Meta-Llama-3.1-8B",
         'mistralai/Mistral-7B-v0.1',

         "norallm/normistral-7b-scratch",
         "norallm/normistral-7b-warm",
         "NorwAI/NorwAI-Mistral-7B", 

         "norallm/normistral-7b-warm-instruct",
         "NorwAI/NorwAI-Mistral-7B-instruct"
         ]

In [18]:
# Create a categorical type with custom ordering
agg_results['model'] = pd.Categorical(
    agg_results['model'],
    categories=model_order,
    ordered=True
)

# Sort the dataframe by the model column
agg_results = agg_results.sort_values('model')

In [19]:
agg_results.to_excel("agg_results_11_01_2025.xlsx")

In [20]:
agg_results

,run,model,mean_acc,mean_acc_ci,mean_Macro_F1,mean_Macro_F1_ci,mean_MCC,mean_MCC_ci
14,14,google/gemma-2-27b-it,0.6053,0.0231,0.5056,0.0154,0.5427,0.0213
12,12,google/gemma-2-2b-it,0.4353,0.0215,0.3262,0.0150,0.3325,0.0187
13,13,google/gemma-2-9b-it,0.6210,0.0180,0.5350,0.0150,0.5513,0.0191
22,22,meta-llama/Llama-2-13b-chat-hf,0.0893,0.0077,0.0612,0.0085,-0.0069,0.0136
21,21,meta-llama/Llama-2-7b-chat-hf,0.0640,0.0071,0.0269,0.0041,0.0008,0.0132
19,19,meta-llama/Meta-Llama-3-8B-Instruct,0.3303,0.0134,0.2644,0.0123,0.2538,0.0167
11,11,meta-llama/Meta-Llama-3.1-8B-Instruct,0.2487,0.0113,0.2180,0.0136,0.1638,0.0193
20,20,mistralai/Mistral-7B-Instruct-v0.1,0.2360,0.0174,0.1733,0.0181,0.0886,0.0233
1,1,RuterNorway/Llama-2-13b-chat-norwegian,0.2327,0.0198,0.1979,0.0110,0.1459,0.0205
0,0,RuterNorway/Llama-2-7b-chat-norwegian,0.1080,0.0114,0.0869,0.0129,0.0280,0.0151
